In [86]:
import pandas as pd
import numpy as np
import re #정규식 지원 모듈
import warnings
warnings.filterwarnings(action='ignore')
import nltk

In [87]:
data = pd.read_csv('movie_dataset/train.csv')
data.head()

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [88]:
data.isna().sum()#간단하게 결측치 확인하기
#텍스트 데이터는 결측값을 어떻게 더해서 나타내는..?

id          0
document    0
label       0
dtype: int64

In [89]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(data) #지정없이 나누면 3750:1250 (7.5:2.5)
train.reset_index(inplace = True) #전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화
val.reset_index(inplace = True) #전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화

In [90]:
train

,index,id,document,label
0,3309,3310,70년대 청춘군상의 엉뚱함과 우울함을 잘 표현한 수작,1
1,71,72,네이버 평점에 심하게 낚였다. 보는 내내 지루해서 끌까말까 망설였는데..,0
2,4359,4360,악!!!ㅋㅋ제리케이다 멋지다!!!!!!,1
3,4490,4491,아무리 박중훈 팬이라도 이 영화는 너무 유치해..정말 아니다.,0
4,3318,3319,적나라한 섹스묘사로 야하긴 하지만 예술성을 칭찬할만하다,1
...,...,...,...,...
3745,1655,1656,나만 당할 수 없지 .. 내 시간 돌리도..,0
3746,2725,2726,알바 ㅡㅡ짜증나네 기분 개잡침 개낚였네그냥 ㅡㅡ,0
3747,1937,1938,"마돈나 좋아라 하지만, 배우로서는 진짜 아니다.",0
3748,4556,4557,흥미진진한 영화였습니다.엔딩이 좀 서둘러 끝난 느낌이네요.,1


In [91]:
val

,index,id,document,label
0,4245,4246,개웃기다. 특히. 팝핀현준. 줘패버리고싶다,0
1,4409,4410,이런류의 애니 원래 좋아하는데 재미가 없어서 스토리가 신선하다고는 못하겠다,0
2,2371,2372,한 번 봤다.그런데.왜.아직도?남은 여운이 나의 마음을 따뜻하게 해준다.,1
3,4468,4469,역션은 전작에 이어 꽤 하지만 스토리가 암담하다,0
4,4861,4862,평점 왜캐 낮아. 공포 영화 톱 10안에 들어감 ㄹㅇ 개무서움,1
...,...,...,...,...
1245,537,538,웨슬리도 이제는 3류배우가 된건가 ...,0
1246,4293,4294,소재는 좋았는데 방향을 완전히 잘못 잡았다.,0
1247,1820,1821,1점도 아깝다 수리수리마수리급 CG,0
1248,2123,2124,진짜 기대하고 봤었는데 완전 실망이었음,0


In [92]:
#데이터 정제

train['preprocessed'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]","") #한글과 공백을 제외하고 모두 제거한 후 train에 preprocessed라는 컬럼으로 저장
train[:5]
#str은 문자열 형태로 객체를 변환하여 돌려주는 함수
#정규식을 사용하여 한글과 공백을 제외하고 제거(replace()는 치환해주는 함수이지만 ""로 치환하면 제거할 수 있음)
#정규식 [^문자들] : [문자들]의 반대로 피해야할 문자들의 집합을 정의

,index,id,document,label,preprocessed
0,3309,3310,70년대 청춘군상의 엉뚱함과 우울함을 잘 표현한 수작,1,년대청춘군상의엉뚱함과우울함을잘표현한수작
1,71,72,네이버 평점에 심하게 낚였다. 보는 내내 지루해서 끌까말까 망설였는데..,0,네이버평점에심하게낚였다보는내내지루해서끌까말까망설였는데
2,4359,4360,악!!!ㅋㅋ제리케이다 멋지다!!!!!!,1,악ㅋㅋ제리케이다멋지다
3,4490,4491,아무리 박중훈 팬이라도 이 영화는 너무 유치해..정말 아니다.,0,아무리박중훈팬이라도이영화는너무유치해정말아니다
4,3318,3319,적나라한 섹스묘사로 야하긴 하지만 예술성을 칭찬할만하다,1,적나라한섹스묘사로야하긴하지만예술성을칭찬할만하다


In [93]:
#혹시 모를 다중 공백도 제거

train['preprocessed'] = train['preprocessed'].str.replace(" +", " ") #다중 공백을 하나의 공백으로 치환(원래 주석에서는 삭제라고 표기)
train[:5] 
# 정규식 [+] : 앞 패턴이 하나 이상이어야 함

,index,id,document,label,preprocessed
0,3309,3310,70년대 청춘군상의 엉뚱함과 우울함을 잘 표현한 수작,1,년대청춘군상의엉뚱함과우울함을잘표현한수작
1,71,72,네이버 평점에 심하게 낚였다. 보는 내내 지루해서 끌까말까 망설였는데..,0,네이버평점에심하게낚였다보는내내지루해서끌까말까망설였는데
2,4359,4360,악!!!ㅋㅋ제리케이다 멋지다!!!!!!,1,악ㅋㅋ제리케이다멋지다
3,4490,4491,아무리 박중훈 팬이라도 이 영화는 너무 유치해..정말 아니다.,0,아무리박중훈팬이라도이영화는너무유치해정말아니다
4,3318,3319,적나라한 섹스묘사로 야하긴 하지만 예술성을 칭찬할만하다,1,적나라한섹스묘사로야하긴하지만예술성을칭찬할만하다


In [94]:
#토큰화(Tokenization)란 말뭉치(Corpus)를 주어진 단위(Token)로 나누는 과정을 의미
# 문장 토큰화와 단어 토큰화가 있음
#문장 토큰화 예시

text = "Hello, nice to meet you. What's your name? Have a nice day! See you soon." # 예시 문장을 정의


In [95]:
from nltk.tokenize import sent_tokenize

print('문장 토큰화 결과 ==>', sent_tokenize(text)) #sent_tokenize() 함수를 통해 문장 토큰화
# 분리 기준으로 "!", "?", "." 등을 설정할 필요 없이 문장 단위로 말뭉치가 분리

문장 토큰화 결과 ==> ['Hello, nice to meet you.', "What's your name?", 'Have a nice day!', 'See you soon.']


In [96]:
#단어 토큰화 예시

from nltk.tokenize import word_tokenize

print('단어 토큰화 결과 ==>', word_tokenize(text)) #word_tokenize() 함수를 통해 단어 토큰화
# "!", "." 등의 구두점도 하나의 단어로 취급되는 것을 확인

단어 토큰화 결과 ==> ['Hello', ',', 'nice', 'to', 'meet', 'you', '.', 'What', "'s", 'your', 'name', '?', 'Have', 'a', 'nice', 'day', '!', 'See', 'you', 'soon', '.']


In [97]:
# 한국어에서의 토큰화
#한국어는 영어와 달리 형태소간의 결합으로 어절이 이루어짐 ex) 철수 -가 책 -을 읽- -었- -다
#따라서 한국어 전용 형태소 분석기가 많음
#KoNLPy라는 패키지의 Okt 라는 형태소 분석기 사용

from konlpy.tag import Okt

han_sentence = "오늘도 열심히 코딩을 해볼까요? 같이 힘내서 자연어 처리 고수가 됩시다! ㅎㅎ"
okt = Okt() # 인스턴스 할당
print("한국어 형태소 분석 결과(어간 추출X) ==>", okt.morphs(han_sentence, stem = False)) # 형태소 단위로 분리
print("한국어 형태소 분석 결과(어간 추출O) ==>", okt.morphs(han_sentence, stem = True)) # 형태소 단위로 분리 후 어간 추출

#okt.morphs() : 형태소 단위로 나눔
#stem은 각 단어에서 어간을 추출하는 옵션

한국어 형태소 분석 결과(어간 추출X) ==> ['오늘', '도', '열심히', '코딩', '을', '해볼까', '요', '?', '같이', '힘내서', '자연어', '처리', '고수', '가', '됩시다', '!', 'ㅎㅎ']
한국어 형태소 분석 결과(어간 추출O) ==> ['오늘', '도', '열심히', '코딩', '을', '해보다', '요', '?', '같이', '힘내다', '자연어', '처리', '고수', '가', '되다', '!', 'ㅎㅎ']


In [98]:
tokenized = [] #토큰화된 데이터를 데이터프레임의 한 컬럼으로 추가할 리스트
for sentence in train['preprocessed']: #전처리된 리뷰들을 하나씩 꺼내옴
    tokens = okt.morphs(sentence, stem = True) # 형태소 분석(어간 추출), 리스트 형태로 저장
    tokenize = " ".join(tokens) #tokens라는 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join(합침)
    tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가
train["tokenized_stem"] = pd.DataFrame(tokenized) #리스트를 데이터프레임으로 변환하여 tokenized_stem라는 컬럼으로 추가

train.head()

,index,id,document,label,preprocessed,tokenized_stem
0,3309,3310,70년대 청춘군상의 엉뚱함과 우울함을 잘 표현한 수작,1,년대청춘군상의엉뚱함과우울함을잘표현한수작,년대 청춘 군 상의 엉뚱하다 우울하다 표현 한 수작
1,71,72,네이버 평점에 심하게 낚였다. 보는 내내 지루해서 끌까말까 망설였는데..,0,네이버평점에심하게낚였다보는내내지루해서끌까말까망설였는데,네이버 평점 에 심하다 낚 이다 보다 지루하다 끌 까맣다 망설이다
2,4359,4360,악!!!ㅋㅋ제리케이다 멋지다!!!!!!,1,악ㅋㅋ제리케이다멋지다,악 ㅋㅋ 제리케이 다 멋지다
3,4490,4491,아무리 박중훈 팬이라도 이 영화는 너무 유치해..정말 아니다.,0,아무리박중훈팬이라도이영화는너무유치해정말아니다,아무리 박중훈 팬 이라도 이영화 는 너무 유치하다 정말 아니다
4,3318,3319,적나라한 섹스묘사로 야하긴 하지만 예술성을 칭찬할만하다,1,적나라한섹스묘사로야하긴하지만예술성을칭찬할만하다,적나라하다 섹스 묘사 로 야하다 예술 성 을 칭찬 하다 만하 다


In [99]:
#품사태깅(POS Tagging)은 주어진 텍스트를 형태소 단위로 나눈 후 각 형태소에 해당 품사를 태깅하여 리스트화 하는 것

#예시
print(okt.pos("오늘도 열심히 재밌는 코딩을 해볼까? 같이 힘내서 자연어 처리 고수들이 됩시다! ㅎㅎ"))
#okt의 pos 함수만을 이용해 토큰화와 품사 태깅을 한번에 할 수 있음
#꼭 필요한 품사(명사 등)를 추출할 때 유용하게 사용

[('오늘', 'Noun'), ('도', 'Josa'), ('열심히', 'Adverb'), ('재밌는', 'Adjective'), ('코딩', 'Noun'), ('을', 'Josa'), ('해볼까', 'Verb'), ('?', 'Punctuation'), ('같이', 'Adverb'), ('힘내서', 'Verb'), ('자연어', 'Noun'), ('처리', 'Noun'), ('고수', 'Noun'), ('들', 'Suffix'), ('이', 'Josa'), ('됩시다', 'Verb'), ('!', 'Punctuation'), ('ㅎㅎ', 'KoreanParticle')]


In [100]:
#okt의 nouns 함수를 사용해 간편하게 명사만 추출
print(okt.nouns("오늘도 열심히 재밌는 코딩을 해볼까? 같이 힘내서 자연어 처리 고수들이 됩시다! ㅎㅎ"))

['오늘', '코딩', '자연어', '처리', '고수']


In [101]:
#영화 리뷰는 명사만으로 긍/부정을 판단하기 어렵기 때문에 임의로 명사, 동사, 형용사, 부사를 추출하여 사용

main_pos = [] #데이터프레임의 새 컬럼이 될 리스트
for sentence in train['document']: #리뷰들을 하나씩 가져옴
    pos = okt.pos(sentence) #형태소 분석을 진행하고 해당 리스트를 pos로 받음
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun","Adverb","Adjective","Verb")] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가
    # if word_pos[1] in ("Noun","Adverb","Adjective","Verb")이면 pos를 돌면서 word_pos[0]에 저장
    main_words_str = " ".join(main_words) #main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join
    main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가
train["main_pos"] = pd.DataFrame(main_pos) #리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가
#main_words_str = "".join(main_words)라고 해서 정확도 0.5xx 까지 떨어짐,,, 즉 띄어쓰기가 분리되지 않은채로 학습이 되니 정확도가 많이 떨어졌음
train.head()

#세 개의 전처리 된 컬럼이 생성되었음
#어떤 전처리 과정이 성능이 잘나올지는 미지수

,index,id,document,label,preprocessed,tokenized_stem,main_pos
0,3309,3310,70년대 청춘군상의 엉뚱함과 우울함을 잘 표현한 수작,1,년대청춘군상의엉뚱함과우울함을잘표현한수작,년대 청춘 군 상의 엉뚱하다 우울하다 표현 한 수작,청춘 상의 엉뚱함과 우울함 잘 표현 수작
1,71,72,네이버 평점에 심하게 낚였다. 보는 내내 지루해서 끌까말까 망설였는데..,0,네이버평점에심하게낚였다보는내내지루해서끌까말까망설였는데,네이버 평점 에 심하다 낚 이다 보다 지루하다 끌 까맣다 망설이다,네이버 평점 심하게 낚였다 보는 내내 지루해서 끌 까말까 망설였는데
2,4359,4360,악!!!ㅋㅋ제리케이다 멋지다!!!!!!,1,악ㅋㅋ제리케이다멋지다,악 ㅋㅋ 제리케이 다 멋지다,악 제리케이 멋지다
3,4490,4491,아무리 박중훈 팬이라도 이 영화는 너무 유치해..정말 아니다.,0,아무리박중훈팬이라도이영화는너무유치해정말아니다,아무리 박중훈 팬 이라도 이영화 는 너무 유치하다 정말 아니다,아무리 박중훈 팬 이 영화 너무 유치해 정말 아니다
4,3318,3319,적나라한 섹스묘사로 야하긴 하지만 예술성을 칭찬할만하다,1,적나라한섹스묘사로야하긴하지만예술성을칭찬할만하다,적나라하다 섹스 묘사 로 야하다 예술 성 을 칭찬 하다 만하 다,적나라한 섹스 묘사 야하긴 예술 칭찬 할 만하


In [102]:
#main_pos 컬럼을 이용하여 모델 학습 진행
#CountVectorizer 사용

X_train = train.main_pos #train 데이터에서 main_pos 추출
y_train = train.label #train 데이터에서 라벨(target) 추출

In [103]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() #countvectorizer 생성
vectorizer.fit(X_train) # countvectorizer 학습
X_train_vec = vectorizer.transform(X_train) # 벡터로 transform, X_traindms 이제 자연어가 아닌 숫자, 벡터 데이터로 이루어져있음
#X_train은 머신러닝 모델의 input 값으로 적절한 데이터의 형식을 갖춤

In [104]:
from sklearn.linear_model import LogisticRegression #모델 불러오기
model = LogisticRegression() #객체에 모델 할당
model.fit(X_train_vec, y_train) # 모델 학습

LogisticRegression()

In [105]:
#검증셋으로 모델 성능 검증
#위와 같은 방식으로 전처리
#미리 나눠놓은 val 사용
val['preprocessed'] = val['document'].str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]","")  #한글과 공백을 제외하고 모두 제거한 후 val에 preprocessed라는 컬럼으로 저장
val['preprocessed'] = val['preprocessed'].str.replace(" +", " ") # 다중 공백을 하나의 공백으로 치환(삭제)

tokenized = []
for sentence in val['preprocessed']:
    tokens = okt.morphs(sentence, stem = True) # stem = True로 설정해 어간 추출
    tokenize = " ".join(tokens) 
    tokenized.append(tokenize)
val["tokenized_stem"] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in val['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words) 
    main_pos.append(main_words_str)
val["main_pos"] = pd.DataFrame(main_pos)

val

,index,id,document,label,preprocessed,tokenized_stem,main_pos
0,4245,4246,개웃기다. 특히. 팝핀현준. 줘패버리고싶다,0,개웃기다 특히 팝핀현준 줘패버리고싶다,개 웃기 다 특히 팝핀현준 주다 패 버리다,개 웃기 특히 팝핀현준 줘 패 버리고싶다
1,4409,4410,이런류의 애니 원래 좋아하는데 재미가 없어서 스토리가 신선하다고는 못하겠다,0,이런류의 애니 원래 좋아하는데 재미가 없어서 스토리가 신선하다고는 못하겠다,이런 류 의 애니 원래 좋아하다 재미 가 없다 스토리 가 신선하다 못 하다,류 애니 원래 좋아하는데 재미 없어서 스토리 신선하다고는 하겠다
2,2371,2372,한 번 봤다.그런데.왜.아직도?남은 여운이 나의 마음을 따뜻하게 해준다.,1,한 번 봤다그런데왜아직도남은 여운이 나의 마음을 따뜻하게 해준다,하다 번 보다 그런데 왜 아직도 남다 여운 이 나 의 마음 을 따뜻하다 해주다,한 번 봤다 왜 아직도 남은 여운 나 마음 따뜻하게 해준다
3,4468,4469,역션은 전작에 이어 꽤 하지만 스토리가 암담하다,0,역션은 전작에 이어 꽤 하지만 스토리가 암담하다,역션 은 전작 에 이다 꽤 하지만 스토리 가 암담하다,역션 전작 이어 꽤 스토리 암담하다
4,4861,4862,평점 왜캐 낮아. 공포 영화 톱 10안에 들어감 ㄹㅇ 개무서움,1,평점 왜캐 낮아 공포 영화 톱 안에 들어감 ㄹㅇ 개무서움,평점 왜캐 낮다 공포 영화 톱 안 에 들어가다 ㄹㅇ 개 무서움,평점 왜캐 낮아 공포 영화 톱 안 들어감 개 무서움
...,...,...,...,...,...,...,...
1245,537,538,웨슬리도 이제는 3류배우가 된건가 ...,0,웨슬리도 이제는 류배우가 된건가,웨슬리 도 이제 는 류 배우 가 되다 건가,웨슬리 이제 류 배우 된 건가
1246,4293,4294,소재는 좋았는데 방향을 완전히 잘못 잡았다.,0,소재는 좋았는데 방향을 완전히 잘못 잡았다,소재 는 좋다 방향 을 완전하다 잘못 잡다,소재 좋았는데 방향 완전히 잘못 잡았다
1247,1820,1821,1점도 아깝다 수리수리마수리급 CG,0,점도 아깝다 수리수리마수리급,점도 아깝다 수리수리마수리 급,점도 아깝다 수리수리마수리
1248,2123,2124,진짜 기대하고 봤었는데 완전 실망이었음,0,진짜 기대하고 봤었는데 완전 실망이었음,진짜 기대하다 보다 완전 실망 이다,진짜 기대하고 봤었는데 완전 실망 이었음


In [106]:
#학습한 모델로 예측할 X를 벡터화

X_val = val.main_pos #validation 데이터에서 전처리된 문서 추출
y_val = val.label #validation 데이터에서 라벨(정답) 추출

X_val_vec = vectorizer.transform(X_val) # train셋으로 fit한 벡터라이저 이용해 transform

In [107]:
#run model
y_pred = model.predict(X_val_vec) #예측값 생성 후 y_pred에 저장
print(y_pred)
#0:negative 1:positive

[0 0 1 ... 0 0 1]


In [108]:
from sklearn import metrics
print('accuracy =', metrics.accuracy_score(y_val, y_pred)) # 실제 정답과 예측값 비교하여 정확도 확인

accuracy = 0.8256


In [109]:
test = pd.read_csv('movie_dataset/test.csv')
test

,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!
2,3,Childhood fantasy
3,4,멋있는 영화입니다. 잊을 수 없는!
4,5,너무 감동적이네요 펑펑 울었습니다
...,...,...
4995,4996,연계도 제대로 안되고 뭔가 부족한느낌이들고 찝찝한영화
4996,4997,내생에 최악의 영화중 하나 졸려서 미칠뻔....
4997,4998,지금까지 나온 드라마 중 최고였습니다. 100점 만점!
4998,4999,영화값으로 엄청난 공연을 보았다 - 이게 웬 횡재 ..


In [110]:
test.isna().sum() #test 데이터 결측치 없음

id          0
document    0
dtype: int64

In [111]:
#테스트 데이터도 train 데이터와 같은 방식으로 전처리 진행
# tokenized_stem은 한글과 공백을 제거하고 남은 데이터에 토큰화 진행한 것(어간 추출)
#main_pos는 명사, 동사, 형용사, 부사만 추출하여 저장한 것

test['preprocessed'] = test['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") #한글과 공백을 제외하고 모두 제거한 후 test에 preprocessed라는 컬럼으로 저장
test['preprocessed'] = test['preprocessed'].str.replace(" +", " ") # 다중 공백을 하나의 공백으로 치환(삭제)

# tokenized_stem
tokenized = []
for sentence in test['preprocessed']:
    tokens = okt.morphs(sentence, stem = True)  # stem = True로 설정해 어간 추출
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
test["tokenized_stem"] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in test['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
test["main_pos"] = pd.DataFrame(main_pos)

test.head()

,id,document,preprocessed,tokenized_stem,main_pos
0,1,시간 때우기 좋은 영화 지루함,시간 때우기 좋은 영화 지루함,시간 때우다 좋다 영화 지루함,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!,훈훈한 정이 느껴지는 영화 가족끼리 드라마 보듯이 보면 딱,훈훈하다 정이 느껴지다 영화 가족 끼리 드라마 보다 보다 딱,훈훈한 정이 느껴지는 영화 가족 끼리 드라마 보듯이 보면 딱
2,3,Childhood fantasy,,,
3,4,멋있는 영화입니다. 잊을 수 없는!,멋있는 영화입니다 잊을 수 없는,멋있다 영화 이다 잊다 수 없다,멋있는 영화 입니다 잊을 수 없는
4,5,너무 감동적이네요 펑펑 울었습니다,너무 감동적이네요 펑펑 울었습니다,너무 감동 적다 펑펑 울다,너무 감동 적이네요 펑펑 울었습니다


In [112]:
#main_pos로 진행

X_test = test.main_pos
X_test_vec = vectorizer.transform(X_test)
pred_test = model.predict(X_test_vec)
print(pred_test)

[0 1 0 ... 1 0 1]


In [113]:
submission = pd.read_csv('movie_dataset/sample_submission.csv')
submission.head()

,id,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [114]:
submission['label'] = pred_test

submission

,id,label
0,1,0
1,2,1
2,3,0
3,4,1
4,5,1
...,...,...
4995,4996,0
4996,4997,0
4997,4998,1
4998,4999,0


In [115]:
submission.to_csv("submission_baseline2.csv", index=False)